<a href="https://colab.research.google.com/github/rafaelfaustini/machinelearning-python/blob/master/breast_cancer_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [217]:
import pandas as pd


In [218]:
labels = ['ID','ClumpThickness', 'UniformityofCellSize', 'UniformityofCellShape', 'MarginalAdhesion', 'SingleEpithelialCellSize', 'BareNuclei', 'BlandChromatin', 'NormalNucleoli', 'Mitoses', 'Class']
df = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data', 
                 names=labels, header=None)

# Limpeza de caracteres não numéricos
cols = df.columns[df.dtypes.eq('object')] # Checa valores que não são int e sim object (strings) e cria uma lista
for col in cols:
  df[col] = pd.to_numeric(df[col], errors='coerce') # Percorro cada um object e transformo em numérico, caso não seja numérico o campo fica com NaN
df = df.dropna() # Exclui as linhas com NaN


In [219]:
x= df.loc[:, 'ClumpThickness':'Mitoses']



In [220]:
y= df.loc[:, 'Class':'Class']
print(y.head())
# 2 câncer benigno -> 0
# 4 câncer maligno -> 1
y = y.replace(to_replace=2, value=0)
y = y.replace(to_replace=4, value=1)
#print(y)

   Class
0      2
1      2
2      2
3      2
4      2


In [221]:
from sklearn.model_selection import train_test_split

In [231]:
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x,y, test_size=0.25) #Ele pega 25% dos registros para testes por isso 0.25
print(x_treinamento.head())

     ClumpThickness  UniformityofCellSize  ...  NormalNucleoli  Mitoses
9                 4                     2  ...               1        1
344               7                     6  ...               5        3
201              10                     8  ...               1        1
486               3                     1  ...               1        1
480               5                     1  ...               1        1

[5 rows x 9 columns]


In [233]:
import keras
from keras.models import Sequential
from keras.layers import Dense
classificador = Sequential()
classificador.add(Dense(units=6, activation= 'relu', 
                        kernel_initializer='random_uniform', input_dim = 9)) # Units: Formula usada (entradas+saida)/2 -> (10+1)/2 -> 5.5 ~= 6 | Input_Dim: Número de previsores de entrada
classificador.add(Dense(units=1, activation= 'sigmoid'))

In [235]:
classificador.compile(optimizer= 'adam', loss='binary_crossentropy',
                      metrics = ['binary_accuracy'])
classificador.fit(x_treinamento, y_treinamento, batch_size=10, epochs=100) # batch_size:calcula de 10, calcula os pesos, calcula mais 10 e calcula os pesos....

Epoch 1/100
512/512 [==============================] - 0s 304us/step - loss: 0.0730 - binary_accuracy: 0.9746
Epoch 2/100
512/512 [==============================] - 0s 123us/step - loss: 0.0707 - binary_accuracy: 0.9766
Epoch 3/100
512/512 [==============================] - 0s 108us/step - loss: 0.0710 - binary_accuracy: 0.9785
Epoch 4/100
512/512 [==============================] - 0s 100us/step - loss: 0.0712 - binary_accuracy: 0.9766
Epoch 5/100
512/512 [==============================] - 0s 120us/step - loss: 0.0715 - binary_accuracy: 0.9766
Epoch 6/100
512/512 [==============================] - 0s 102us/step - loss: 0.0730 - binary_accuracy: 0.9766
Epoch 7/100
512/512 [==============================] - 0s 97us/step - loss: 0.0713 - binary_accuracy: 0.9746
Epoch 8/100
512/512 [==============================] - 0s 96us/step - loss: 0.0710 - binary_accuracy: 0.9766
Epoch 9/100
512/512 [==============================] - 0s 98us/step - loss: 0.0718 - binary_accuracy: 0.9746
Epoch 10/100


In [251]:

previsoes = classificador.predict(x_teste)
previsoes = (previsoes > 0.5)
from sklearn.metrics import confusion_matrix, accuracy_score
acuraciaSklearn = accuracy_score(y_teste, previsoes)
matrix = confusion_matrix(y_teste, previsoes)
print(matrix) # (0,0) Benignos classificados como 0; [Correto](0,1)Benignos Classificados como 1; [Errado](1,0) Malignos classificados como 0; [Errado](0,1) Malignos classificados como 1 [Correto];

acuraciaKeras = classificador.evaluate(x_teste, y_teste)
print("Acuracias - Keras: {0} %, SkLearn {1} %".format(acuracia*100, resultado[1]*100))


[[101   2]
 [  2  66]]
171/171 [==============================] - 0s 57us/step
Acuracias - Keras: 97.6608187134503 %, SkLearn 97.66082167625427 %
